In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
import pyomo.environ as pyo
from pyomo.environ import *
import sys

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def generate_negatively_correlated_covariance_matrix(n):
    # Generate a random correlation matrix with negative correlations
    correlation_matrix = np.random.uniform(low=-0.9, high=-0.1, size=(n, n))
    np.fill_diagonal(correlation_matrix, 1)  # Set diagonal elements to 1
    correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2  # Ensure symmetry

    # Ensure the correlation matrix is valid for Cholesky decomposition
    cholesky_success = False
    while not cholesky_success:
        try:
            # Perform Cholesky decomposition
            cholesky_matrix = np.linalg.cholesky(correlation_matrix)
            cholesky_success = True
        except np.linalg.LinAlgError:
            # If Cholesky decomposition fails, adjust the correlation matrix slightly
            correlation_matrix += np.eye(n) * 0.01  # Adjust diagonal elements slightly

    # Generate random standard deviations
    #standard_deviations = np.random.uniform(0.5, 2, size=n)
    
    variance_outsideopt = np.random.uniform(30,40)
    variance_products = np.random.uniform(0.5,2,size=n-1)
    standard_deviations = np.hstack((variance_outsideopt,variance_products))
    # Construct diagonal matrix from standard deviations
    diagonal_matrix = np.diag(standard_deviations)
    #diagonal_matrix = np.eye(n)

    # Calculate the covariance matrix using Cholesky decomposition and diagonal matrix
    covariance_matrix = cholesky_matrix @ diagonal_matrix @ cholesky_matrix.T

    return covariance_matrix



In [5]:
def is_positive_semidefinite(matrix):
    """
    Check if a matrix is positive semidefinite.
    
    Args:
        matrix (numpy.ndarray): The matrix to check.
        
    Returns:
        bool: True if the matrix is positive semidefinite, False otherwise.
    """
    # Check if the matrix is symmetric
    if not np.allclose(matrix, matrix.T):
        return False
    
    # Check if all eigenvalues are non-negative
    eigvals, _ = np.linalg.eig(matrix)
    return np.all(eigvals >= 0)


In [6]:
def collection_distribution_prob_neg(n,collection):
    # generate mean 
    #mu = np.random.uniform(-2,2,n)
    mu_0 = np.random.uniform(6,10)
    mu_1n = np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    # generate covariance matrix with positive correlation
    neg_cov = generate_negatively_correlated_covariance_matrix(n)
    if is_positive_semidefinite(neg_cov): 
        sample_size = 10000
        utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    else:
        print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 
        
        
        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [7]:
def whole_instance_generation(n,whole_collection,whole_offertimes):
    ## generate LCMNL instances based on the assortment collection infomation
    # step 1: randomly generate LCMNL parameters
    # 1.1 the number of classes in LCMNL, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mnl
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCMNL instance with the above parameters of LCMNL
    true_instance = collection_distribution_prob_neg(n,whole_collection)
    # step 3: generate multinomial samples based on lcmnl instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [8]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [9]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [10]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [11]:
# check if the collection are nested
def check_subsets(lists):
    # Convert all lists to sets
    sets = [set(lst) for lst in lists]
    
    # Check subsets
    for i in range(len(sets) - 1):
        if not sets[i + 1].issubset(sets[i]):
            return False
    return True

# Check and print the result
result = check_subsets(all_full_assortment_index)
print("All subsequent lists are subsets of the previous one:", result)

All subsequent lists are subsets of the previous one: True


In [12]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [13]:
# generate instances that satisfy for assortment offertimes is >=20
## generate 50 random full instances 
# the observations with at least 20 times are >= 1e-3
pred_full_instance = []
pred_full_samples = []
#collection_20,offertimes_20,assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[0])

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)
    
    ''' condition = False
    for x in range(len(all_full_collections[0])):
        for y in all_full_collections[0][x]:
            if curr_choice_collection[y][x]<1e-11:
                condition = True
    while condition:
        curr_whole_instance,curr_purchase_samples = whole_instance_generation(jd_collection,jd_offertimes)
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

        condition = False
        for x in range(len(all_full_collections[0])):
            for y in all_full_collections[0][x]:
                if curr_choice_collection[y][x] < 1e-11:
                    condition = True
    if condition == True:
        print('instance_generation error')
    else:

            
        pred_full_instance.append(curr_whole_instance)
        pred_full_samples.append(curr_purchase_samples) '''

generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [14]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [15]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
        # re-check if all choice probability are >= 1e-3
        condition = False
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                if curr_choice_collection[y][x] < 1e-3:
                    condition = True
        if condition == True:
            print('instance_generation error')
        
        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation err

In [16]:
# train-test split
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(all_full_collections[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(all_full_collections[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(all_full_collections[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = all_full_instances[i][j][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = all_full_lb[i][j][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = all_full_ub[i][j][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(all_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(all_full_collections[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = all_full_instances[i][j][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        ''' df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''
        
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [17]:
pd.DataFrame(all_train_instances[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.713816,0.75641,0.75122,0.764444,0.758882,0.769231,0.684211,0.784314,0.695652,0.733871,...,0.60,0.769231,0.707317,0.708333,0.62500,0.844444,0.869565,0.857143,0.800000,0.764706
1,0.286184,0.00000,0.00000,0.000000,0.000000,0.000000,0.210526,0.196078,0.282609,0.177419,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.24359,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.00000,0.24878,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.000000,...,0.25,0.038462,0.146341,0.250000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.00000,0.00000,0.235556,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,...,0.15,0.000000,0.000000,0.000000,0.03125,0.044444,0.043478,0.000000,0.000000,0.000000
5,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,0.000000,...,0.00,0.192308,0.000000,0.000000,0.34375,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.00000,0.00000,0.000000,0.241118,0.000000,0.000000,0.000000,0.000000,0.088710,...,0.00,0.000000,0.146341,0.000000,0.00000,0.111111,0.000000,0.000000,0.114286,0.000000
7,0.000000,0.00000,0.00000,0.000000,0.000000,0.230769,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.086957,0.000000,0.085714,0.000000
8,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.041667,0.00000,0.000000,0.000000,0.142857,0.000000,0.235294


In [18]:
pd.DataFrame(all_test_instance[0][0])

,0,1,2,3,4
0,0.772567,0.750,0.746269,0.718593,0.787879
1,0.000000,0.000,0.000000,0.000000,0.000000
2,0.000000,0.000,0.000000,0.000000,0.000000
3,0.000000,0.125,0.000000,0.000000,0.000000
4,0.000000,0.000,0.000000,0.000000,0.000000
5,0.227433,0.000,0.119403,0.135678,0.000000
6,0.000000,0.000,0.134328,0.000000,0.030303
7,0.000000,0.125,0.000000,0.000000,0.000000
8,0.000000,0.000,0.000000,0.145729,0.181818


In [19]:
pd.DataFrame(all_train_lb[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.671804,0.619981,0.716619,0.708293,0.729904,0.718617,0.511387,0.622648,0.505216,0.622469,...,0.382568,0.537289,0.592157,0.447895,0.384769,0.692785,0.672445,0.671514,0.610210,0.712838
1,0.244173,0.000000,0.000000,0.000000,0.000000,0.000000,0.058950,0.040021,0.096254,0.081119,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.107160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.214180,0.000000,0.000000,0.000000,-0.008840,0.000000,0.000000,0.000000,...,0.057816,-0.067405,0.056883,0.001891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.179404,0.000000,0.000000,0.000000,-0.034890,0.000000,0.000000,...,-0.008479,0.000000,0.000000,0.000000,-0.055088,-0.041789,-0.075884,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.038616,0.000000,...,0.000000,-0.024654,0.000000,0.000000,0.108067,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.212140,0.000000,0.000000,0.000000,0.000000,0.017037,...,0.000000,0.000000,0.056883,0.000000,0.000000,-0.020394,0.000000,0.000000,-0.036672,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.180155,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.077966,0.000000,-0.047111,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.072830,0.000000,0.000000,0.000000,-0.042772,0.000000,0.183427


In [20]:
pd.DataFrame(all_train_ub[0][0])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.755827,0.892840,0.785820,0.820596,0.787860,0.819845,0.857034,0.945980,0.886089,0.845273,...,0.817432,1.001172,0.822477,0.968771,0.865231,0.996104,1.066686,1.042772,0.989790,0.816573
1,0.328196,0.000000,0.000000,0.000000,0.000000,0.000000,0.362103,0.352136,0.468963,0.273719,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.380019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.283381,0.000000,0.000000,0.000000,0.219366,0.000000,0.000000,0.000000,...,0.442184,0.144328,0.235800,0.498109,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.291707,0.000000,0.000000,0.000000,0.074105,0.000000,0.000000,...,0.308479,0.000000,0.000000,0.000000,0.117588,0.130678,0.162841,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.082095,0.000000,...,0.000000,0.409269,0.000000,0.000000,0.579433,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.270096,0.000000,0.000000,0.000000,0.000000,0.160382,...,0.000000,0.000000,0.235800,0.000000,0.000000,0.242617,0.000000,0.000000,0.265244,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.281383,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.251879,0.000000,0.218540,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.156164,0.000000,0.000000,0.000000,0.328486,0.000000,0.287162


In [21]:
all_test_collection[0][0]

[[0, 5], [0, 3, 7], [0, 5, 6], [0, 5, 8], [0, 6, 8]]

In [22]:
def expectation_step(v_values,data,collections,L,offer_times):
    n,m = data.shape

    # Pyomo model
    model = pyo.ConcreteModel()

    # Variables
    model.w = pyo.Var(range(L), domain=pyo.NonNegativeReals,bounds=(0.001,1))

    # Objective function
    model.obj = pyo.Objective(
        expr=sum(sum(offer_times[i] * data[j, i] * pyo.log(
            sum(model.w[l] * np.exp(v_values[l, j]) / sum(np.exp(v_values[l, k]) for k in collections[i]) for l in range(L))
        ) for j in collections[i]) for i in range(m)),
        sense=pyo.maximize
    )

    # Constraint: Sum of weights equals 1
    model.sum_weights_constraint = pyo.Constraint(expr=sum(model.w[l] for l in range(L)) == 1)

    # Solve the optimization problem
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)
    
    # Print optimization status
    #print("Optimization Status:", results.solver.status)
    if results.solver.status == pyo.SolverStatus.ok:
        #print('Optimal solution found!')
       
        #model.pprint()
    
        w_values = [model.w[l]() for l in range(L)]
        w_values = np.array(w_values)
        
        ''' print("Variable w =", [ model.w[l]() for l in range(L)])
        print("Objective Value =", model.obj()) '''
        
        return w_values,pyo.value(model.obj())
    else:
        print('No solution found or an error occurred.')


In [23]:
def maximization_step(weights,data,collections,offer_times):
    # Given data
    L = len(weights)
    n,m = data.shape
    
    # Create a ConcreteModel
    model = pyo.ConcreteModel()

    # Define decision variables
    model.v = pyo.Var(range(L), range(n), bounds=(-100, 100))

    # Define the objective function
    model.obj = pyo.Objective(
        expr=sum(sum(data[j][i] * offer_times[i] * pyo.log(sum(weights[l] * pyo.exp(model.v[l, j]) / sum(pyo.exp(model.v[l, k]) for k in collections[i]) for l in range(L))) for j in collections[i]) for i in range(m)),
        sense=pyo.maximize
    )
    
    # preference parameter of outside option 
    model.outsideopt_constraint = pyo.ConstraintList()
    for l in range(L):
        model.outsideopt_constraint.add(model.v[l,0] == 0)

    # Solve the model
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)

    # Print optimization status
    #print("Optimization Status:", results.solver.status)
    if results.solver.status == pyo.SolverStatus.ok:
        #print('Optimal solution found!')
        #model.write('lcmnl_maximizatioin.mps', io_options={'symbolic_solver_labels': True})
        #model.pprint()
    
        v_values = [[model.v[l, i]() for i in range(n)] for l in range(L)]
        v_values = np.array(v_values)

        # Print variable values and objective value
        ''' print("Variable v =", [[model.v[l, i]() for i in range(n)] for l in range(L)])
        print("Objective Value =", model.obj()) '''
        
        return v_values,model.obj()
    else:
        print('No solution found or an error occurred.')
    


In [24]:
# EM algorithm for solving the MLE of LC-MNL
def expectation_maximization(data, collections,offer_times, num_class, max_iterations):
    
    weights = np.ones(num_class)/num_class
    v_values,ll = maximization_step(weights,data,collections,offer_times)
    #print('initial loglikelihood values',ll)
    #print('intitial preference parameters',v_values)

    for i in range(max_iterations):
        curr_ll = ll
        weights,ll = expectation_step(v_values,data,collections,num_class,offer_times)
        #print(f'the loglikelihood after expectation at iteration {i} is {ll}')
        v_values,ll = maximization_step(weights,data,collections,offer_times)
        #print(f'the loglikelihood after maximization at iteration {i} is {ll}')
        
        ''' if ll - curr_ll < 0.001:
            print(f'{ll} at current iteration and {curr_ll} at last iteration  ')
            return weights, v_values,ll '''
        
    return   weights, v_values,ll
        

In [25]:
def collection_distribution_lcmnl(n,collection,num_mix,weights,parameters_v):
    
    
    collection_distribution = np.zeros((n,len(collection)))
    
    for i in range(len(collection)): 
        curr_total_v = np.zeros(num_mix) # the denominator for each class
        
        for j in range(len(collection[i])): 
            for k in range(num_mix):
                curr_total_v[k] = curr_total_v[k] + math.exp(parameters_v[k][collection[i][j]])
        #print('checking the weights of denominators',curr_total_v)
        
        for j in range(len(collection[i])):
            for k in range(num_mix):
                collection_distribution[collection[i][j]][i] = collection_distribution[collection[i][j]][i] + weights[k]*math.exp(parameters_v[k][collection[i][j]])/curr_total_v[k]
    
    return collection_distribution

In [26]:
## prediction for the current testing collection of assortments 
def lcmnl_prediction(curr_test_instance,curr_test_collection,price,weights,v_values,n):
    ''' true assortment ranking '''
    true_choice_prob = [] 
    true_revenue = []
    
    lcmnl_revenue = []
    lcmnl_sales = []
    
    # lc- mnl outofsample choice probability using EM estimators 
    
    estimated_oos_prob = collection_distribution_lcmnl(n,curr_test_collection,len(weights),weights,v_values)
    
    
    for k in range(len(curr_test_collection)):

        # true revenue and true choice probabilities for current testing assortment
        curr_revenue = np.dot(curr_test_instance[:,k],price)
        true_revenue.append(curr_revenue)
        true_choice_prob.append(curr_test_instance[:,k])
        curr_lcmnl_rev = np.dot(estimated_oos_prob[:,k],price)
        
        lcmnl_sales.append(estimated_oos_prob[:,k])
        lcmnl_revenue.append(curr_lcmnl_rev)
        print(f'revenue of assortment {curr_test_collection[k]} is {curr_lcmnl_rev}')
        
        ## record the runtime result
        
        
    return true_revenue,true_choice_prob,lcmnl_revenue,lcmnl_sales

In [27]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [28]:
## record representability of all instances

## record the kendall tau distance between the predicted ranking of lcmnl and the truth
all_lcmnl_distance_list = []  

## record the true revenue of the predicted best assortment
all_best_true_rev_list = []
all_best_rev_lcmnl_list = []  

## record the difference in best assortment
all_lcmnl_best_rev_diff = []

## record all the true and predicted revenues
all_true_revenue_list = []
all_lcmnl_revenue_list = []

all_true_prob_list = [] 
all_lcmnl_sales_list = []

all_true_ranking = []
all_lcmnl_ranking = []

## record the runtime of lcmnl prediction
all_lcmnl_pred_runtime = []

num_class = 10
max_iterations = 50

for i in range(len(all_full_collections)):
       
    # define as a container for quantity of the same collection size 
    # The following lists are of the same size as the instance size
    
    true_best_rev_collection = [] # a container for groundtruth best reveue 
    lcmnl_best_rev_collection = [] # a container for the best revenue predicted by robust lcmnl
    
    true_ranking_collection = [] # a container for groundtruth reveue ranking 
    lcmnl_ranking_collection = [] # a container for ranking by robust lcmnl
    
    lcmnl_distance_collection = [] # a container for kendall tau distance between true ranking and the ranking by robust lcmnl
   
    
    true_choice_prob_collection = []
    lcmnl_sales_frac_collection = []

    true_revenue_collection = []
    lcmnl_revenue_collection = []
    
    lcmnl_runtime_collection = []
    
    for j in range(pred_instance_size[i]):
        
        print(f'testing for offertimes {pred_offer_times_list[i]}, {j}th instance')
        
        starttime = time.time()
        weights, v_values,ll = expectation_maximization(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j], num_class, max_iterations)
        #lcmnl_choiceprob_100 = collection_distribution_lcmnl(n,collection_100,num_class,weights,v_values)
        lcmnl_mle_runtime = time.time()-starttime
        
        true_revenue,true_choice_prob,lcmnl_revenue,lcmnl_sales = lcmnl_prediction(all_test_instance[i][j],all_test_collection[i][j],price,weights,v_values,n)
        # the following lists are of the same size as number of testing assortments
        
        ll_timelst = [lcmnl_mle_runtime]* len(all_test_collection[i][j])
        #tolruntime = [x + y for x, y in zip(ll_timelst, lcmnl_runtime)]
        
        true_revenue_collection.append(true_revenue)
        true_choice_prob_collection.append(true_choice_prob)
        
        lcmnl_revenue_collection.append(lcmnl_revenue)   
        lcmnl_sales_frac_collection.append(lcmnl_sales)
        lcmnl_runtime_collection.append(ll_timelst)
         
        # true assortment ranking for test instance [i][j]
        curr_true_rank = np.argsort(-np.array(true_revenue))  
        true_ranking_collection.append(curr_true_rank)
        # true best assortment revenue for test instance [i][j]
        true_best_rev_collection.append(true_revenue[curr_true_rank[0]])
        
        # lb mdm assortment ranking for test instance [i][j] 
        curr_lcmnl_rank  = np.argsort(-np.array(lcmnl_revenue)) 
        lcmnl_ranking_collection.append(curr_lcmnl_rank)
        lcmnl_best_rev_collection.append(true_revenue[curr_lcmnl_rank[0]])
        
        
        #  lcmnl kendallTau Distance for current instance [i][j]
        curr_lcmnl_dist = kendall_tau_distance(curr_true_rank, curr_lcmnl_rank)
        lcmnl_distance_collection.append(curr_lcmnl_dist)
    
    
    # for each test instance [i][j] kendall tau distance is a number
    all_lcmnl_distance_list.append(lcmnl_distance_collection)
    
    # for each test instance [i][j] best revenue is a number
    all_best_true_rev_list.append(true_best_rev_collection) 
    all_best_rev_lcmnl_list.append(lcmnl_best_rev_collection)
    
    # for each test instance [i][j] best revenue difference is a number
    all_lcmnl_best_rev_diff.append((np.array(true_best_rev_collection) - np.array(lcmnl_best_rev_collection))/ np.array(true_best_rev_collection))
    
    # for each test instance [i][j], true revenue is a list of the same size as the number of testing assortments
    all_true_revenue_list.append(true_revenue_collection)
    
    all_lcmnl_revenue_list.append(lcmnl_revenue_collection)
    
    all_true_prob_list.append(true_choice_prob_collection)
    all_lcmnl_sales_list.append(lcmnl_sales_frac_collection)
 
    # for each test instance [i][j], true ranking is a list of the same size as the number of testing assortments
    all_true_ranking.append(true_ranking_collection)
    all_lcmnl_ranking.append(lcmnl_ranking_collection)

    ## for each test instance [i][j], runtime is a list of the same size as the number of testing assortments
    all_lcmnl_pred_runtime.append(lcmnl_runtime_collection)

testing for offertimes 20, 0th instance
revenue of assortment [0, 5] is 0.3343154482403657
revenue of assortment [0, 3, 7] is 0.1542987224382949
revenue of assortment [0, 5, 6] is 0.2686486663841331
revenue of assortment [0, 5, 8] is 0.42416794384719014
revenue of assortment [0, 6, 8] is 0.2827898860657366
testing for offertimes 20, 1th instance
revenue of assortment [0, 3] is 0.13568639900675059
revenue of assortment [0, 4] is 0.2925456991970776
revenue of assortment [0, 1, 4] is 0.43582877621634486
revenue of assortment [0, 3, 4] is 0.18525570233010272
revenue of assortment [0, 5, 6] is 0.2541242243187481
testing for offertimes 20, 2th instance
revenue of assortment [0, 1, 8] is 0.5062320061487657
revenue of assortment [0, 2, 6] is 0.15224591671690504
revenue of assortment [0, 3, 4] is 0.4628812981722232
revenue of assortment [0, 4, 5] is 0.5522110908406548
revenue of assortment [0, 5, 8] is 0.536901379218955
testing for offertimes 20, 3th instance
revenue of assortment [0, 1] is 0.1

In [29]:
## adding details 1
for i in range(len(pred_offer_times_list)):
    df_lcmnl_details1 = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'lcmnl_dist':all_lcmnl_distance_list[i],
                                    'true_best_rev':all_best_true_rev_list[i],'lcmnl_best_rev':all_best_rev_lcmnl_list[i],
                                    'lcmnl_best_rev_diff':all_lcmnl_best_rev_diff[i]})
    df_lcmnl_details1.to_csv('prediction/lcmnl/revenue_prediction/details1/'+str(pred_offer_times_list[i])+'.csv')

In [30]:
## adding details 2 about ranking, revenue, choice probability
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        
        true_rank = all_true_ranking[i][j]
        lcmnl_rank = all_lcmnl_ranking[i][j]
        df_rank = pd.DataFrame({'true_rank':true_rank,'lcmnl_rank':lcmnl_rank})
        df_rank.to_csv('prediction/lcmnl/revenue_prediction/details2/ranking/offertimes'+str(pred_offer_times_list[i])+'/rank_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        true_rev = all_true_revenue_list[i][j]
        lcmnl_rev = all_lcmnl_revenue_list[i][j]
        df_rev = pd.DataFrame({'true_rev':true_rev,'lcmnl_rev':lcmnl_rev})
        df_rev.to_csv('prediction/lcmnl/revenue_prediction/details2/revenue/offertimes'+str(pred_offer_times_list[i])+'/rev_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
    
        lcmnl_runtime = all_lcmnl_pred_runtime[i][j]
        df_runtime = pd.DataFrame({'lcmnl_runtime':lcmnl_runtime})
        df_runtime.to_csv('prediction/lcmnl/revenue_prediction/details2/runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        ## using sales fraction prediction
        ''' ## record the choice probability prediction interval
        true_prob = all_true_prob_list[i][j]
        df_true_prob = pd.DataFrame(true_prob)
        
        lcmnl_sales = all_lcmnl_sales_list[i][j]
        df_lcmnl_sales = pd.DataFrame(lcmnl_sales).T
 
        df_true_prob.to_csv('prediction/lcmnl/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_lcmnl_sales.to_csv('prediction/lcmnl/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/lcmnl/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''

In [31]:
## avg prediction lcmnl runtime 
all_lcmnl_pred_avg_runtime = []

for i in range(len(pred_offer_times_list)):
    avg_lcmnl_pred_runtime_collection = []
    
    for j in range(pred_instance_size[i]): 
        avg_lcmnl_pred_runtime_collection.append(np.mean(np.array(all_lcmnl_pred_runtime[i][j]))) 
        
    all_lcmnl_pred_avg_runtime.append(np.mean(np.array(avg_lcmnl_pred_runtime_collection)))

In [32]:
avg_lcmnl_dist = []
avg_lcmnl_best_rev_diff = []

for i in range(len(pred_offer_times_list)):
    
    avg_lcmnl_dist.append(np.mean(all_lcmnl_distance_list[i]))
    avg_lcmnl_best_rev_diff.append(np.mean(all_lcmnl_best_rev_diff[i]))
    

df_lcmnl_jd_summary = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'avg_lcmnl_dist':avg_lcmnl_dist,'avg_lcmnl_best_rev_diff':avg_lcmnl_best_rev_diff,
                                   'avg_lcmnl_pred_runtime':all_lcmnl_pred_avg_runtime})
df_lcmnl_jd_summary.to_csv('prediction/lcmnl/jd_lcmnl_summary.csv')

In [33]:
df_lcmnl_jd_summary

,offertimes,avg_lcmnl_dist,avg_lcmnl_best_rev_diff,avg_lcmnl_pred_runtime
0,20,1.72,0.054730,36.717284
1,30,1.02,0.054697,37.674805
2,40,0.48,0.061706,40.190941
3,50,0.60,0.073121,34.937027
4,60,0.14,0.048631,36.208400


In [34]:

## record the limit loss of all instances
all_lcmnl_limit_loss = []
all_lcmnl_limit_prob = []
all_lcmnl_limit_runtime = []

for i in range(len(pred_offer_times_list)):

    
    ## define container for lcmnl limit 
    lcmnl_limit_loss_collection =[]
    lcmnl_limit_runtime_collection =[]
    lcmnl_limit_probability_collection =[]
    
    for j in range(pred_instance_size[i]):
        starttime = time.time()
        weights, v_values,ll = expectation_maximization(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i], num_class, max_iterations)
        lcmnl_limit_prob = collection_distribution_lcmnl(n,all_full_collections[i],num_class,weights,v_values)
        lcmnl_limit_runtime = time.time()-starttime
        
        lcmnl_limit_loss = 0
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                lcmnl_limit_loss = lcmnl_limit_loss + all_full_offertimes[i][x]*all_full_instances[i][j][y][x]*np.abs(all_full_instances[i][j][y][x] -lcmnl_limit_prob[y][x])
                 
        lcmnl_limit_probability_collection.append(lcmnl_limit_prob)
        lcmnl_limit_loss_collection.append(lcmnl_limit_loss)
        lcmnl_limit_runtime_collection.append(lcmnl_limit_runtime)

    
    ## for each train instance [i][j] limit loss result is a number
    all_lcmnl_limit_loss.append(lcmnl_limit_loss_collection)
    all_lcmnl_limit_runtime.append(lcmnl_limit_runtime_collection)
    
    ## for each train instance [i][j] limit probability is a matrix
    all_lcmnl_limit_prob.append(lcmnl_limit_probability_collection)

In [35]:
## adding limit results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    df_lcmnl_loss = pd.DataFrame({'ins_idx':ins_idx,'lcmnl_loss':all_lcmnl_limit_loss[i],'avg_loss':all_lcmnl_limit_loss[i]/sum(all_full_offertimes[i]),  'lcmnl_limit_time':all_lcmnl_limit_runtime[i]})
    df_lcmnl_loss.to_csv('limit/lcmnl/limit/'+str(pred_offer_times_list[i])+'.csv')
    
    for j in range(pred_instance_size[i]):
        df_limit_prob = pd.DataFrame(all_lcmnl_limit_prob[i][j])
        df_limit_prob.to_csv('limit/lcmnl/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [49]:
all_avg_loss = []
for i in range(len(pred_offer_times_list)):
    all_avg_loss.append(all_lcmnl_limit_loss[i]/np.sum(all_full_offertimes[i]))

In [50]:


avg_total_limit_loss = []
avg_toal_limit_loss_se = []
avg_lcmnl_limit_runtime = []

#avg_loss = []
avg_loss_se = []

for i in range(len(pred_offer_times_list)):

    
    avg_total_limit_loss.append(np.mean(all_lcmnl_limit_loss[i]))
    avg_lcmnl_limit_runtime.append(np.sum(all_lcmnl_limit_runtime[i]))
    avg_toal_limit_loss_se.append(np.std(all_lcmnl_limit_loss[i])/np.sqrt(len(all_lcmnl_limit_loss[i])))
    
    #avg_loss = np.mean(all_avg_loss[i])
    avg_loss_se.append(np.std(all_avg_loss[i])/np.sqrt(len(all_avg_loss[i])))

In [51]:
avg_loss = []
for i in range(len(avg_total_limit_loss)):
    avg_loss.append(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

In [52]:
df_lcmnl_limit = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_lcmnl_loss':avg_total_limit_loss,'total_lcmnl_loss_se':avg_toal_limit_loss_se,
                                   'avg_lcmnl_loss':avg_loss,'avg_lcmnl_loss_se':avg_loss_se,
                                   'avg_lcmnl_limit_runtime':avg_lcmnl_limit_runtime
                                   })
df_lcmnl_limit.to_csv('limit/lcmnl/lcmnl_limit_summary.csv')
df_lcmnl_limit

,offertimes,total_lcmnl_loss,total_lcmnl_loss_se,avg_lcmnl_loss,avg_lcmnl_loss_se,avg_lcmnl_limit_runtime
0,20,54.790824,2.536538,0.007158,0.000331,2114.440923
1,30,40.816556,1.467249,0.005421,0.000195,2241.195410
2,40,31.705640,3.887736,0.004307,0.000528,2646.293190
3,50,23.891906,5.484987,0.003323,0.000763,2481.704641
4,60,16.202362,3.527019,0.002288,0.000498,1892.705337


In [53]:
for i in range(len(avg_total_limit_loss)):
    print(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

0.007157521156695845
0.005421245354330352
0.004307246355276026
0.003322935449023263
0.0022878228724909004
